In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]=""

In [14]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import pandas as pd
from tqdm import tqdm
import torch
import os
import numpy as np
from itertools import product

In [16]:
%autoreload 2
from huggingmolecules import MatConfig, MatFeaturizer, MatModel
from huggingmolecules import GroverConfig, GroverFeaturizer, GroverModel
from huggingmolecules import RMatConfig, RMatFeaturizer, RMatModel
from models import StrippedMatModel, StrippedGroverModel, StrippedRMatModel

In [63]:
%autoreload 2
from utils import *
from datasets import *
import predict_procedure as pp
import embedding_procedure as ep

In [72]:
%%bash

python embedding_procedure.py \
    --model_name Graphormer \
    --dataset_name HIA_Hou \
    --max_samples 10

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'
Found local copy...
Loading...
Done!
Found local copy...


Processing HIA_Hou with Graphormer


INFO: Processing HIA_Hou with Graphormer
100%|██████████| 10/10 [00:00<00:00, 11.94it/s]


Processing HIA_Hou with Graphormer took 0.0m


INFO: Processing HIA_Hou with Graphormer took 0.0m


In [6]:
# from mat import MolecularAttentionTransformer

# model = MolecularAttentionTransformer()
# model.predict(["CN1CC[C@]23C(=O)C[C@@H]4C(=CCO[C@H]5CC(=O)N(c6ccccc62)[C@@H]3[C@@H]54)C1"]) ## tooks 11m 4s (!)

In [7]:
# group = admet_group(path = 'data/')
# benchmark = group.get("CYP2D6_Substrate_CarbonMangels")

# dataset = benchmark["test"]

# print(dataset[dataset.Drug == "CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C"].to_markdown())

# Output:
# |     | smiles                             |   Y |
# |----:|:-----------------------------------|----:|
# |  58 | CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C |   1 |
# | 226 | CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C |   0 |else 'regression'

In [74]:
import pathlib

calculated = set(map(str, pathlib.Path("embeddings/").glob("*/*.zip")))

pairs = product(get_dataset_names(), ["Graphormer"])

with open("embedding_params.txt", "w") as f:
    for dataset_name, model_name in pairs:
        if f"embeddings/{dataset_name}/{model_name}.zip" not in calculated:
            print(dataset_name, model_name, file=f)


In [75]:
import pathlib
done = set(map(str, pathlib.Path("results/").glob("*.pkl")))

pairs = list(product(get_classification_names(), ["Graphormer"]))
calculated = list(map(lambda s: tuple(str(s).replace(".", "/").split("/")[1:-1]), pathlib.Path("embeddings/").glob("*/*.zip")))

params = list(product(list(set(pairs)), ["linear", "forest"]))


with open("predict_params.txt", "w") as f:
    for (dataset_name, model_name), head_model in params:
        if f"results/{dataset_name}_{model_name}_{head_model}.pkl" not in done:
            print(dataset_name, model_name, head_model, file=f)
            

In [36]:
import pathlib
done = set(map(str, pathlib.Path("results/").glob("*.pkl")))

pairs = list(product(get_dataset_names(), get_embedder_names()))
calculated = list(map(lambda s: tuple(str(s).replace(".", "/").split("/")[1:-1]), pathlib.Path("embeddings/").glob("*/*.zip")))

params = list(product(list(set(pairs) & set(calculated)), ["ridge","forest"]))


with open("predict_params.txt", "w") as f:
    for (dataset_name, model_name), head_model in params:
        if f"results/{dataset_name}_{model_name}_{head_model}.pkl" in done and dataset_name in get_classification_names():
            print(dataset_name, model_name, head_model, file=f)

In [19]:
dataset_name, model_name = "HIA_Hou", "molbert"

df, _ = get_dataset(dataset_name)

model = get_embedder(model_name)

ep.procedure(dataset_name, df, model_name, model, max_samples=10)

Found local copy...
Loading...
Done!
Found local copy...


Processing HIA_Hou with molbert


INFO: Processing HIA_Hou with molbert
100%|██████████| 10/10 [00:04<00:00,  2.34it/s]

Processing HIA_Hou with molbert took 0.1m



INFO: Processing HIA_Hou with molbert took 0.1m


In [ ]:
for dataset_name in get_dataset_names():
    get_dataset(dataset_name)[0].reset_index()[["index", "smiles"]].rename(columns={"index": "chembl_id", "smiles": "canonical_smiles"}).to_csv(f"smiles_txt/{dataset_name}.txt", sep='\t', index=False)


In [ ]:
for dataset_name in get_dataset_names():
    os.system(f"python SELFormer/generate_selfies.py --smiles_dataset=smiles_txt/{dataset_name}.txt --selfies_dataset=selfies/{dataset_name}.csv")

In [ ]:
with open("selformer_params.txt", "w") as f:
    for dataset_name in get_dataset_names():
        if len(get_dataset(dataset_name)[0]) < 10000:
            f.write(f"{dataset_name}\n")
    

In [8]:
a = pd.read_csv("selformer_embeddings/Half_Life_Obach.csv").rename(columns={"chembl_id": "index"})
a

,index,sequence_embeddings
0,0,"[1.2678825855255127, -0.04146221652626991, -0...."
1,1,"[0.8832750916481018, -0.09156806021928787, -0...."
2,2,"[1.2266165018081665, -0.035363443195819855, -0..."
3,3,"[1.1679654121398926, 0.12847484648227692, -0.4..."
4,4,"[1.3297243118286133, 0.20555377006530762, -0.3..."
...,...,...
662,662,"[1.3313795328140259, -0.08630513399839401, -0...."
663,663,"[1.2012218236923218, 0.008925149217247963, -0...."
664,664,"[1.079952359199524, -0.17504577338695526, -0.5..."
665,665,"[1.0526909828186035, 0.13809537887573242, -0.4..."


In [27]:
from transformers import GraphormerForGraphClassification

In [39]:
from ogb.graphproppred import PygGraphPropPredDataset

# dataset = PygGraphPropPredDataset(name="ogbg-molesol")
dataset = get_dataset("HIA_Hou", True)[0].graph

Found local copy...
Loading...
Done!
Found local copy...


In [30]:
dataset

0      [(x, [tensor([1., 0., 0., 0., 0., 0., 0., 0., ...
1      [(x, [tensor([1., 0., 0., 0., 0., 0., 0., 0., ...
2      [(x, [tensor([1., 0., 0., 0., 0., 0., 0., 0., ...
3      [(x, [tensor([1., 0., 0., 0., 0., 0., 0., 0., ...
4      [(x, [tensor([0., 1., 0., 0., 0., 0., 0., 0., ...
                             ...                        
573    [(x, [tensor([1., 0., 0., 0., 0., 0., 0., 0., ...
574    [(x, [tensor([1., 0., 0., 0., 0., 0., 0., 0., ...
575    [(x, [tensor([0., 0., 1., 0., 0., 0., 0., 0., ...
576    [(x, [tensor([1., 0., 0., 0., 0., 0., 0., 0., ...
577    [(x, [tensor([0., 1., 0., 0., 0., 0., 0., 0., ...
Name: graph, Length: 578, dtype: object

In [31]:
model_checkpoint = "clefourrier/graphormer-base-pcqm4mv2" # pre-trained model from which to fine-tune

model = GraphormerForGraphClassification.from_pretrained(
    model_checkpoint,
)

In [32]:
from transformers.models.graphormer.collating_graphormer import preprocess_item, GraphormerDataCollator

In [33]:
import torch

In [40]:
a = dataset[0]
a

Data(x=[10, 39], edge_index=[2, 20])

In [38]:
a = dataset[0]
a

Data(edge_index=[2, 68], edge_attr=[68, 3], x=[32, 9], y=[1, 1], num_nodes=32)

In [41]:
b = preprocess_item(a)
b

KeyError: 'num_nodes'

In [28]:
len({**b}["input_nodes"])

32

In [49]:
{**b, "labels": None}["labels"]

In [73]:
model.classifier = torch.nn.Identity()

In [55]:
GraphormerDataCollator()([{**b}])

{'attn_bias': tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]),
 'attn_edge_type': tensor([[[[   0,    0,    0],
           [   2,  514, 1026],
           [   0,    0,    0],
           ...,
           [   0,    0,    0],
           [   0,    0,    0],
           [   0,    0,    0]],
 
          [[   2,  514, 1026],
           [   0,    0,    0],
           [   2,  514, 1026],
           ...,
           [   0,    0,    0],
           [   0,    0,    0],
           [   0,    0,    0]],
 
          [[   0,    0,    0],
           [   2,  514, 1026],
           [   0,    0,    0],
           ...,
           [   0,    0,    0],
           [   2,  514, 1026],
           [   0,    0,    0]],
 
          ...,
 
          [[   0,    0,    0],
           [   0,    0,    0],
     

In [74]:
def preprocess(molecules):
    result = GraphormerDataCollator()([{**preprocess_item(molecule)} for molecule in molecules])
    result["labels"] = None
    return result

preprocess([a])

{'attn_bias': tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]),
 'attn_edge_type': tensor([[[[   0,    0,    0],
           [   2,  514, 1026],
           [   0,    0,    0],
           ...,
           [   0,    0,    0],
           [   0,    0,    0],
           [   0,    0,    0]],
 
          [[   2,  514, 1026],
           [   0,    0,    0],
           [   2,  514, 1026],
           ...,
           [   0,    0,    0],
           [   0,    0,    0],
           [   0,    0,    0]],
 
          [[   0,    0,    0],
           [   2,  514, 1026],
           [   0,    0,    0],
           ...,
           [   0,    0,    0],
           [   2,  514, 1026],
           [   0,    0,    0]],
 
          ...,
 
          [[   0,    0,    0],
           [   0,    0,    0],
     

In [75]:
with torch.no_grad():
    res = model.forward(**preprocess([a])).logits.flatten()
res.logits.flatten()

tensor([-1.5532e-01, -1.1019e-01, -1.2027e-01, -1.0760e-01, -1.3930e-01,
        -8.2543e-02, -2.5050e-01, -3.3259e-01, -2.0304e-01, -9.5992e-02,
        -2.7759e-01,  6.8701e+00, -1.7186e-01, -1.3394e-01, -5.9249e-02,
        -8.0273e-02, -3.1434e-01, -7.0930e-02, -0.0000e+00, -1.3310e-01,
        -2.5710e-01, -1.8664e-01, -3.1301e-01, -2.1712e-01,  8.7888e-01,
        -7.4379e-02, -2.0731e-01, -1.7346e-01,  2.3891e-01, -8.4378e-02,
        -1.9693e-01, -1.6561e-01,  5.1644e-01, -1.7614e-01,  1.2662e+00,
        -8.5366e-02, -6.1126e-02,  1.5683e+00, -1.9905e-01, -2.7775e-01,
        -1.4630e-01, -2.7886e-01,  1.5578e+00, -9.6614e-02,  2.0428e-01,
        -1.8741e-02, -1.1139e-01, -6.9050e-02, -7.7757e-02,  3.4922e-01,
        -1.7356e-01, -1.1356e-01, -8.1240e-02, -6.2769e-02, -1.9859e-01,
         4.5292e-01, -9.7752e-02, -1.2888e-01, -3.0186e-01, -2.9533e-01,
        -1.9562e-01, -3.0877e-01, -1.1360e-01,  1.8262e-01, -1.0039e-01,
        -3.1856e-01, -1.7098e-01, -1.0643e-01,  7.8

In [58]:
torch.IntTensor([32, 1]).size()[:2]

torch.Size([2])

In [70]:
b.input_nodes = torch.Tensor(b.input_nodes)

In [71]:
model.forward(**b)

IndexError: too many indices for tensor of dimension 2

In [2]:
from datasets import load_dataset

dataset = load_dataset("OGB/ogbg-molesol")

AttributeError: module 'datasets' has no attribute 'load_dataset'

In [4]:
datasets.__dir__()

['__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__file__',
 '__cached__',
 '__builtins__',
 'pd',
 'PygGraphPropPredDataset',
 'logging',
 'time',
 'torch',
 'MatConfig',
 'MatFeaturizer',
 'MatModel',
 'GroverConfig',
 'GroverFeaturizer',
 'GroverModel',
 'GroverBatchEncoding',
 'RMatConfig',
 'RMatFeaturizer',
 'RMatModel',
 'StrippedMatModel',
 'StrippedGroverModel',
 'StrippedRMatModel',
 'Iterator',
 'Callable',
 'Tuple',
 'tqdm',
 'dc',
 'Chem',
 'chain',
 'ADME',
 'admet_group',
 'load_ogb_dataset',
 'load_tdc_dataset',
 'get_ogb_regression_names',
 'get_ogb_classification_names',
 'get_ogb_names',
 'get_tdc_regression_names',
 'get_tdc_classification_names',
 'get_tdc_names',
 'get_regression_names',
 'get_classification_names',
 'get_dataset_names',
 'get_dataset']

In [ ]:
dataset[0]

In [74]:
GraphormerDataCollator()(b)

KeyError: 0

In [35]:
model(GraphormerDataCollator([b]))

TypeError: GraphormerForGraphClassification.forward() missing 6 required positional arguments: 'input_edges', 'attn_bias', 'in_degree', 'out_degree', 'spatial_pos', and 'attn_edge_type'

In [4]:
from transformers.models.graphormer.collating_graphormer import preprocess_item

In [13]:
b["edge_index"]

tensor([[ 0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,
          9, 10, 10, 11, 11, 12, 12, 13, 11, 14, 14, 15, 15, 16, 16, 17, 17, 18,
         18, 19,  9, 20, 20, 21, 20, 22, 22, 23, 22, 24, 24, 25,  4, 26, 26, 27,
         26, 28, 28, 29, 28, 30, 30, 31, 19, 14, 24,  7, 30,  2],
        [ 1,  0,  2,  1,  3,  2,  4,  3,  5,  4,  6,  5,  7,  6,  8,  7,  9,  8,
         10,  9, 11, 10, 12, 11, 13, 12, 14, 11, 15, 14, 16, 15, 17, 16, 18, 17,
         19, 18, 20,  9, 21, 20, 22, 20, 23, 22, 24, 22, 25, 24, 26,  4, 27, 26,
         28, 26, 29, 28, 30, 28, 31, 30, 14, 19,  7, 24,  2, 30]])

In [68]:
from tdc.chem_utils import MolConvert
converter = MolConvert(src = 'SMILES', dst = 'PyG')
converter('C').x
# Data(edge_index=[2, 68], edge_attr=[68, 3], x=[32, 9], y=[1, 1], num_nodes=32)

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
         0., 0., 0.]])

In [69]:
from ogb.utils import smiles2graph
{**smiles2graph('C'), "x": converter('C').x, "y": np.array([0])}

{'edge_index': array([], shape=(2, 0), dtype=int64),
 'edge_feat': array([], shape=(0, 3), dtype=int64),
 'node_feat': array([[5, 0, 4, 5, 4, 0, 2, 0, 0]]),
 'num_nodes': 1,
 'x': tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
          0., 0., 0.]]),
 'y': array([0])}

In [64]:
get_dataset("HIA_Hou", True)[0]

Found local copy...
Loading...
Done!
Found local copy...


,smiles,Y,graph
0,CC(=O)Nc1ccccc1,1,"{'edge_index': [[0, 1, 1, 2, 1, 3, 3, 4, 4, 5,..."
1,CC(=O)Nc1nnc(S(N)(=O)=O)s1,1,"{'edge_index': [[0, 1, 1, 2, 1, 3, 3, 4, 4, 5,..."
2,Cc1c(N(C)C)c(=N)n(-c2ccccc2)n1C,1,"{'edge_index': [[0, 1, 1, 2, 2, 3, 3, 4, 3, 5,..."
3,COc1ccccc1OCCNC[C@@H](O)c1ccc(C)cc1S(N)(=O)=O,1,"{'edge_index': [[0, 1, 1, 2, 2, 3, 3, 4, 4, 5,..."
4,NS(=O)(=O)c1cc2c(cc1C(F)(F)F)N[C@H](Cc1ccccc1)...,1,"{'edge_index': [[0, 1, 1, 2, 1, 3, 1, 4, 4, 5,..."
...,...,...,...
573,CC(=CCO)/C=C/C=C(C)\C=C\C1=C(C)CCCC1(C)C,1,"{'edge_index': [[0, 1, 1, 2, 2, 3, 3, 4, 1, 5,..."
574,CCN(CC)[C@H](C)C(=O)c1ccccc1,1,"{'edge_index': [[0, 1, 1, 2, 2, 3, 3, 4, 2, 5,..."
575,OC[C@@H]1O[C@@H](OC[C@@H]2O[C@@H](O[C@]3(CO)O[...,0,"{'edge_index': [[0, 1, 1, 2, 2, 3, 3, 4, 4, 5,..."
576,COC(=O)C1=C(C#N)NC(C)=C(C(=O)OC(C)C)[C@H]1c1cc...,1,"{'edge_index': [[0, 1, 1, 2, 2, 3, 2, 4, 4, 5,..."
